In [ ]:

!pip install unidecode
!pip install inflector
!pip install pickle5
import nltk
nltk.download('punkt')
import warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=256382 sha256=b88de6c8b4e5c34cf3042bf5484339a6b32d2adc8f71343247196c1a8d257910
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/evaluar_modelos_max')

In [ ]:
import os
import numpy as np
import pandas as pd
import models.clean_txt as clean_txt
import models.multipr as multipr
import models.pymes as pymes
import models.sentimiento as sentimiento
import models.lasso as lasso
from tqdm import tqdm
import models.nn as nn
stemmed_words = lasso.stemmed_words

from glob import glob


In [ ]:
root = '/content/drive/MyDrive/evaluar_modelos_max/'

file = glob('/content/drive/MyDrive/BID_Capital/Mexico/Textos/2022_data.parquet')

df = pd.read_parquet(file)

df["title"] = df.title.astype(str).fillna("")

texts = list(df['text'])
titles = list(df['title'])


print('---------------clean text--------------')
texts_cl = multipr.mpfunc(clean_txt.clean_txt, texts, step=30)
titles_cl = multipr.mpfunc(clean_txt.clean_txt, titles, step=30)
print('---------------done clean text--------------')



print('---------------is pyme--------------')
pymes_text = multipr.mpfunc(pymes.ispyme, texts_cl, step=30)
pymes_title = multipr.mpfunc(pymes.ispyme, titles_cl, step=30)
print('---------------done is pyme--------------')



print('---------------sentiment--------------')
df_w = pd.read_csv(root + '/models/diccionario_sentimiento.csv')
w2k, w2d = sentimiento.define_words_sentiment(df_w)
sent = sentimiento.score(w2k, w2d)
title_sent = multipr.mpfunc(sent.check_text_apply_score, titles_cl, step=30)
text_sent = multipr.mpfunc(sent.check_text_apply_score, texts_cl, step=30)
print('---------------done sentiment--------------')

---------------clean text--------------


100%|██████████| 4454/4454 [00:28<00:00, 155.22it/s]


---------------done clean text--------------
---------------is pyme--------------


100%|██████████| 4454/4454 [00:16<00:00, 278.12it/s]


---------------done is pyme--------------
---------------sentiment--------------


100%|██████████| 4454/4454 [05:23<00:00, 13.76it/s]


---------------done sentiment--------------


In [ ]:
print('---------------lasso sector--------------')
w2v = lasso.w2v(root + '/models/lasso_models/text_vectorizer_sectors.pickle')
sentences =  w2v.cutted_for_vectorize(texts_cl, step=150)
modelos = [root + '/models/lasso_models/' + f
           for f in os.listdir(root + '/models/lasso_models') if 'vectorizer' not in f]
modelos = lasso.lasso_class(modelos)
prediction_lasso = []
for s in tqdm(sentences):
    X = w2v.transform(s)
    predict_lasso, keys = [], []
    for k, v in modelos.models_dict.items():
        predict_lasso.append(v.predict_proba(X)[:,1].reshape((len(s), 1)))
        keys.append(k)
    predict_lasso = pd.DataFrame(np.concatenate(predict_lasso, axis=1))
    predict_lasso.columns = [k.replace(root + '/models/lasso_models/', '') for k in keys]
    prediction_lasso.append(predict_lasso)
    del X
del k, v, keys, predict_lasso, s, modelos, sentences
prediction_lasso = pd.concat(prediction_lasso, axis=0, ignore_index=True)
print('---------------done lasso sector--------------')



print('---------------nn sector--------------')
w2v_nn = nn.w2v(root + '/models/nn_model/text_vectorizer_layer_sectors.pickle')
sentences =  w2v_nn.cutted_for_vectorize(texts_cl, step=150)
modelo = nn.model(root + '/models/nn_model/cnn_model.h5')
prediction_nn = []
for s in tqdm(sentences):
    X = w2v_nn.transform(s)
    p = modelo.predict(X)
    prediction_nn.append(p)
    del X, s, p
del modelo, sentences
prediction_nn = pd.concat(prediction_nn, axis=0, ignore_index=True)
prediction_nn.columns = ['Pr_' + x for x in prediction_nn.columns]
prediction_nn = prediction_nn.astype(float)
print('---------------done nn sector--------------')



print('---------------consolidate the data--------------')
df['text_pyme'] = pymes_text
df['title_pyme'] = pymes_title
df['pyme'] = (df['text_pyme'] + df['title_pyme']).astype(bool)
df['text_sentiment'] = text_sent
df = pd.concat([df, prediction_lasso], axis=1)
df = pd.concat([df, prediction_nn], axis=1)
print('---------------done consolidate the data--------------')



---------------lasso sector--------------


100%|██████████| 891/891 [07:55<00:00,  1.87it/s]


---------------done lasso sector--------------
---------------nn sector--------------


  0%|          | 0/891 [00:00<?, ?it/s]

5/5 [==============================] - 5s 8ms/step


...

100%|█████████▉| 890/891 [34:14<00:02,  2.32s/it]

4/4 [==============================] - 0s 11ms/step


100%|██████████| 891/891 [34:16<00:00,  2.31s/it]

---------------done nn sector--------------
---------------consolidate the data--------------


---------------done consolidate the data--------------


In [ ]:
path = '/content/drive/MyDrive/evaluar_modelos_max/Textos clasificados/'+file[0].replace('.parquet', '_class.parquet').split('/')[-1]
df.to_parquet(path)

In [ ]:
file

['/content/drive/MyDrive/BID_Capital/Mexico/Textos/2022_data.parquet']